In [ ]:
import matplotlib.pyplot as plt

%matplotlib inline

# https://stackoverflow.com/questions/17687213/how-to-obtain-the-same-font-style-size-etc-in-matplotlib-output-as-in-latex
params = {'text.usetex' : True,
          'font.size'   : 10, # the book seems to be in 10pt, change if needed
          'font.family' : 'lmodern',
          }

plt.rcParams.update(params)
default_size_inches = (3.54,3.54) 
plt.rcParams['figure.figsize'] = default_size_inches

In [ ]:
[ k for k in plt.rcParams.keys() if 'font' in k ]

In [ ]:
import pyomo.environ as pyo
import pyomo
pyomo.__version__

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('./code/')
import draw

In [ ]:
draw.SetOutputPath( '/work in progress/MO book/results' )

# Chapter one

In [ ]:
import sympy, math, numpy as np, sys

x  = sympy.Symbol('x')

# now pi is a symbol, just like x
pi = sympy.Symbol('pi')

# we redefine h using the same right-hand-side code as before, 
# but now with x and pi as symbols
h = (pi*x**2 + 500)/(2*pi*x+50)

# to have the drivative on the symbol pi we need it from the new version of h
hprime = sympy.diff( h, x )

solution = sympy.solveset( sympy.diff( h, x ), x )
solution

In [ ]:
def Preety( formula ):
    from sympy import latex
    from IPython.display import display, Math
    display( Math( latex( formula ) ) )

Preety( h )
Preety( sympy.simplify( h ) )
Preety( hprime )
Preety( sympy.simplify( hprime ) )

In [ ]:
Preety( solution )

s = max(solution.subs( pi, math.pi ).evalf())

print(s)

In [ ]:
def Plot( h, s=None, start=0, stop=20, width=18, height=8, file_name=None ):
    with plt.rc_context({'figure.figsize': (width,height)}): 
        plt.rcParams['figure.figsize'] = (width,height)

        plt.grid()
        plt.xlabel(r'$x$')
        plt.ylabel(r'$h(x)$')

        plt.xticks(np.arange(start, stop+1, step=1))

        x = sympy.Symbol('x')
        f = sympy.lambdify( x, h.subs( pi, math.pi ) )

        import numpy
        x = numpy.linspace(start=start,stop=stop,num=100) 
        y = f(x)

        plt.plot(x,y,label='$h(x)='+sympy.latex(h)+'$',linewidth=3)
        if s is None:
            x = numpy.linspace(start=start,stop=stop,num=stop-start+1) 
            y = f(x)
            plt.plot(x,y, 'bo', label='some points', markersize=8)
        else:
            plt.plot(s,f(s), 'ro', label='$x^*$ optimum', markersize=8)

        plt.legend()

        if file_name is not None:
            plt.savefig( draw._output_path+file_name, bbox_inches='tight', pad_inches=0 )

        plt.show()

In [ ]:
Plot( h, None, 0, 20, 8, 5, 'AliceSome.pdf' )

In [ ]:
Plot( h, s, 0, 20, 6, 3, 'AliceOptimum.pdf' )

# Chapter two

In [ ]:
def SimpleDraw( model ):
    with plt.rc_context({'figure.figsize': (8,6)}):
        return draw.Draw( model, isolines=True, file_name=model.name+'.pdf' )

In [ ]:
def CreateBIM():
    m    = pyo.ConcreteModel('BIM')
    
    m.x1 = pyo.Var( within=pyo.NonNegativeReals )
    m.x2 = pyo.Var( within=pyo.NonNegativeReals )

    m.obj       = pyo.Objective( sense= pyo.maximize
                               , expr = 12*m.x1 + 9*m.x2 )

    m.silicon   = pyo.Constraint(expr =    m.x1          <= 1000)
    m.germanium = pyo.Constraint(expr =             m.x2 <= 1500)
    m.plastic   = pyo.Constraint(expr =    m.x1 +   m.x2 <= 1750)
    m.copper    = pyo.Constraint(expr =  4*m.x1 + 2*m.x2 <= 4800)
    
    return m

In [ ]:
SimpleDraw( CreateBIM() )

In [ ]:
basicfeasiblesolutions = draw.Draw( CreateBIM(), 'BuildingMicrochips.pdf' )

In [ ]:
basicfeasiblesolutions.rename(columns={'x1': '$x_1$', 'x2': '$x_2$'}).astype(int).to_latex(draw._output_path+'chips.tex')
if 'google.colab' in sys.modules:
    import os
    from google.colab import files
    files.download( 'chips.tex' )

# Chapter three

The first version was a copy of [the model on this deck](http://web.tecnico.ulisboa.pt/mcasquilho/compute/_linpro/TaylorB_module_c.pdf]).

Below several versions of possible models.

In [ ]:
def CreateBBaExample():
    model    = pyo.ConcreteModel('BBa')
    
    model.x1 = pyo.Var( within=pyo.NonNegativeReals )
    model.x2 = pyo.Var( within=pyo.NonNegativeReals )

    model.obj = pyo.Objective( sense= pyo.maximize
                                    , expr = 2*model.x1 + 3*model.x2 )

    model.c1  = pyo.Constraint(expr =  2*model.x1 +  1*model.x2 <=  10)
    model.c2  = pyo.Constraint(expr =  3*model.x1 +  6*model.x2 <=  40)
    
    return model

In [ ]:
draw.Draw( CreateBBaExample(), integer=True, isolines=True, file_name=CreateBBaExample().name+'.pdf', title='First B\&B example' )

In [ ]:
sol,root = draw.BB( CreateBBaExample(), solver='gurobi_direct' )

In [ ]:
draw.ToTikz(root, 'BBa.tex', fig_only=True )

In [ ]:
draw.DrawBB(root, 'BB.pdf')

In [ ]:
draw.DotExporter(root).to_picture(draw._output_path+'BBplain.pdf')

In [ ]:
def CreateBBbExample():
    m    = pyo.ConcreteModel('BBb')
    
    m.x1 = pyo.Var( within=pyo.NonNegativeReals )
    m.x2 = pyo.Var( within=pyo.NonNegativeReals )

    m.obj= pyo.Objective( sense= pyo.maximize
                        , expr =  1*m.x1 + 2*m.x2 )

    m.c1 = pyo.Constraint(expr = -4*m.x1 + 5*m.x2 <= 11)
    m.c2 = pyo.Constraint(expr =  5*m.x1 - 2*m.x2 <=  9)
    
    return m

In [ ]:
draw.Draw( CreateBBbExample(), integer=True, isolines=False, file_name=None, title='test' )

In [ ]:
sol,root = draw.BB( CreateBBbExample(), solver='gurobi_direct' )

In [ ]:
draw.ToTikz(root, 'BBb.tex', fig_only=True )

In [ ]:
draw.DotExporter(root).to_dotfile(draw._output_path+'test.dot')

In [ ]:
draw.DrawBB(root,'BBBook.pdf')

In [ ]:
def CreateBIMmodified():
    m    = pyo.ConcreteModel('BIMmodified')
    
    m.x1 = pyo.Var( within=pyo.NonNegativeReals )
    m.x2 = pyo.Var( within=pyo.NonNegativeReals )

    m.obj       = pyo.Objective( sense= pyo.maximize
                               , expr = 12*m.x1 + 9*m.x2 )

    m.silicon   = pyo.Constraint(expr =    m.x1          <= 900 )
    m.germanium = pyo.Constraint(expr =             m.x2 <= 1350)
    m.plastic   = pyo.Constraint(expr =    m.x1 +   m.x2 <= 1801)
    m.copper    = pyo.Constraint(expr =  4*m.x1 + 2*m.x2 <= 4903)
    
    return m

In [ ]:
sol,root = draw.BB( CreateBIMmodified(), solver='gurobi_direct', draw_integer=False, xlim=(-50,1050), ylim=(-50,1550) )

In [ ]:
draw.ToTikz(root, 'BIMmodified.tex', fig_only=True )

In [ ]:
def CreateBIMperturbed():
    m    = pyo.ConcreteModel('BIMperturbed')
    
    m.x1 = pyo.Var( within=pyo.NonNegativeReals )
    m.x2 = pyo.Var( within=pyo.NonNegativeReals )

    m.obj       = pyo.Objective( sense= pyo.maximize
                               , expr = 12*m.x1 + 9*m.x2 )

    m.silicon   = pyo.Constraint(expr =    m.x1                <= 1000 )
    m.germanium = pyo.Constraint(expr =                   m.x2 <= 1500)
    m.plastic   = pyo.Constraint(expr =    m.x1    +      m.x2 <= 1750)
    m.copper    = pyo.Constraint(expr =  4.04*m.x1 + 2.02*m.x2 <= 4800)
    
    return m

In [ ]:
sol,root = draw.BB( CreateBIMperturbed(), solver='gurobi_direct', draw_integer=False, xlim=(-50,1050), ylim=(-50,1550) )

In [ ]:
draw.ToTikz(root, 'BIMperturbed.tex', fig_only=True )

In [ ]:
draw.Draw( CreateBIM(), integer=False, isolines=True, file_name=None, title='First B\&B example' )

In [ ]:
nx,ny = 6,5
F = { (2,1), (3,1), (4,1), (2,2), (3,2), (4,2), (3,3), (4,3), (3,4) }

import itertools  
points = list(itertools.product( range(0,nx+1), range(0,ny+1) ) )
feasible   = [ p for p in points if p in F ]
infeasible = [ p for p in points if not p in F ]
if infeasible:
    plt.plot( *zip(*infeasible), 'ro', zorder=2, markersize=9)
if feasible:
    plt.plot( *zip(*feasible), 'bo', zorder=2, markersize=9)
    
def Pol( coord, style, alpha, width ):
    coord.append(coord[0]) #repeat the first point to create a 'closed loop'
    plt.plot(*zip(*coord),style, alpha=alpha, linewidth=width, zorder=1) 

Pol( [ (1.6,.5), (2.5,5), (4.5,3.5), (4,.5) ], 'g-', 1, 2 )
Pol( [ (2,1), (2,2), (2.5, 3.5), (3.5, 4.5), (4,3), (4,1) ], 'm--', .8, 3 )
Pol( [ (2,1), (2,2), (3,4), (4,3), (4,1) ], 'c-.', 1, 3 )

ax = plt.gca()

# Hide the right and top spines
for position in ['left','right','top','bottom']:
    ax.spines[position].set_visible(False)
    
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)

plt.savefig( draw._output_path+'3regions.pdf', bbox_inches='tight', pad_inches=0 )
plt.show()